In [139]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import os

In [140]:
os.getcwd()

'/Users/jackkalsched/Desktop/Independent Projects/SMT-Data-Challenge-2025'

In [141]:
#Filtering game_info for plays that have a runner on base
## Assuming that we are only accounting for plays with a runner on base, looking to advance to third/home.
runners_df = pd.read_csv('game_info/QEA/ACN/year_1/day_070/game_info.csv')
runners_df['runner_on'] = (runners_df['first_baserunner'].notnull()) | (runners_df['second_baserunner'].notnull()) | (runners_df['third_baserunner'].notnull()) 
runners_df = runners_df[runners_df['runner_on'] == True]

runners_df


,game_str,home_team,away_team,at_bat,play_per_game,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,runner_on
6,y1_d070_ACN_QEA,QEA,ACN,NaN,7,top,QEA-0175,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,True
7,y1_d070_ACN_QEA,QEA,ACN,2.0,8,top,QEA-0175,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,True
8,y1_d070_ACN_QEA,QEA,ACN,2.0,9,top,QEA-0175,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,True
9,y1_d070_ACN_QEA,QEA,ACN,2.0,11,top,QEA-0175,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,True
10,y1_d070_ACN_QEA,QEA,ACN,2.0,12,top,QEA-0175,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,y1_d070_ACN_QEA,QEA,ACN,72.0,300,top,QEA-0108,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,True
295,y1_d070_ACN_QEA,QEA,ACN,72.0,301,top,QEA-0108,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-1180,ACN-1623,NaN,NaN,True
296,y1_d070_ACN_QEA,QEA,ACN,73.0,302,top,QEA-0108,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-1180,ACN-1623,NaN,NaN,True
297,y1_d070_ACN_QEA,QEA,ACN,73.0,303,top,QEA-0108,QEA-0120,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-1180,NaN,ACN-1623,NaN,True


How do we determine plays in which the runner is trying to reach the next base? 

First, the ball needs to be hit in play.

In [142]:
batted_balls = pd.read_csv('game_events/QEA/ACN/year_1/day_070/game_events.csv')


#Filtering for plays that aren't just pitch -> throw back to pitcher -> end play
prev2 = batted_balls['event_code'].shift(2)
prev1 = batted_balls['event_code'].shift(1)
curr  = batted_balls['event_code']
next1 = batted_balls['event_code'].shift(-1)
next2 = batted_balls['event_code'].shift(-2)

pattern1 = (curr == 5) & (prev1 == 2) & (prev2 == 1)   
pattern2 = (curr == 1) & (next1 == 2) & (next2 == 5)   
pattern3 = (curr == 2) & (prev1 == 1) & (next1 == 5) 
pattern4 = (curr == 5) & (prev1 == 4) #foulball (hit ball, end of play)
pattern5 = (curr == 4) & (next1 == 5) #foulball
pattern6 = (curr == 1)

batted_balls['non_play'] = pattern1 | pattern2 | pattern3 | pattern4 | pattern5 | pattern6

#filter for play_id's that have event_code 4 in them (batted ball)
bbip = batted_balls.groupby('play_id')['event_code'].apply(lambda x: (x == 4).any())
filtered_batted_balls = batted_balls[batted_balls['play_id'].isin(bbip[bbip].index)]

filtered_batted_balls = filtered_batted_balls[filtered_batted_balls['non_play'] == False]

filtered_batted_balls.head(15)



,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,non_play
16,y1_d070_ACN_QEA,6,1,6,112287,10,4,False
17,y1_d070_ACN_QEA,6,1,6,114187,255,16,False
18,y1_d070_ACN_QEA,6,1,6,114937,255,16,False
19,y1_d070_ACN_QEA,6,1,6,115437,255,16,False
20,y1_d070_ACN_QEA,6,1,6,115887,255,16,False
21,y1_d070_ACN_QEA,6,1,6,116237,7,2,False
22,y1_d070_ACN_QEA,6,1,6,117737,7,3,False
23,y1_d070_ACN_QEA,6,1,6,119237,6,2,False
24,y1_d070_ACN_QEA,6,1,6,120087,0,5,False
47,y1_d070_ACN_QEA,14,2,14,282987,10,4,False


In [143]:
#merging both datasets
runners_on_bbip = pd.merge(runners_df, filtered_batted_balls, on = ['game_str', 'play_per_game'], how = 'inner')

#re-organizing columns
runners_on_bbip = runners_on_bbip[['game_str', 'play_id', 'timestamp', 'home_team', 'away_team', 'at_bat_y', 'play_per_game', 'top_bottom_inning',
                                   'pitcher', 'catcher', 'first_base', 'second_base', 'shortstop', 'third_base', 'left_field', 
                                   'center_field', 'right_field', 'batter', 'first_baserunner', 'second_baserunner', 'third_baserunner',
                                   'player_position', 'event_code']]
runners_on_bbip = runners_on_bbip.rename(columns = {'at_bat_y': 'at_bat'})


#Number of filtered plays in this particular game
print(runners_on_bbip['play_id'].nunique())

#df for balls in play with runners on 
runners_on_bbip

20


,game_str,play_id,timestamp,home_team,away_team,at_bat,play_per_game,top_bottom_inning,pitcher,catcher,...,third_base,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,player_position,event_code
0,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,QEA-0232,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,10,4
1,y1_d070_ACN_QEA,14,283087,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,QEA-0232,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,255,16
2,y1_d070_ACN_QEA,14,284737,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,QEA-0232,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,1,2
3,y1_d070_ACN_QEA,14,286187,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,QEA-0232,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,1,3
4,y1_d070_ACN_QEA,14,287087,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,QEA-0232,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,y1_d070_ACN_QEA,297,9939137,QEA,ACN,71,297,top,QEA-0108,QEA-0120,...,QEA-0232,QEA-0364,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,8,2
140,y1_d070_ACN_QEA,297,9940287,QEA,ACN,71,297,top,QEA-0108,QEA-0120,...,QEA-0232,QEA-0364,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,8,3
141,y1_d070_ACN_QEA,297,9942637,QEA,ACN,71,297,top,QEA-0108,QEA-0120,...,QEA-0232,QEA-0364,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,255,16
142,y1_d070_ACN_QEA,297,9943237,QEA,ACN,71,297,top,QEA-0108,QEA-0120,...,QEA-0232,QEA-0364,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,5,2


I guess filtering down for plays where the runner "takes the extra base", and then starting to calculate probabilities?

### Thought process for filtering for runners taking the extra base:

- For second to home/first to home, it will be hard to determine programmatically that an event happened where a runner from second attempted to take two bases
    - Will probably require player-pos and ball-tracking data to make assumptions
    - Also can be used to assume out/safe calls
- For first to third, can filter by the same runner_id appearing sequentially in the prior play's "first_baserunner" column and then in the next play's "third_baserunner" column


## First to Third Filtering

In [144]:
extra_base = runners_on_bbip

next_baserunner = extra_base['third_baserunner'].shift(-1)

#Redefining prev, curr, and next variables from previous cells
curr = extra_base['event_code']
prev1 = extra_base['event_code'].shift(1)
next1 = extra_base['event_code'].shift(-1)

extra_base['first_to_third'] = (curr == 5) & (next1 == 4) & (next_baserunner == extra_base['first_baserunner'])

#Making first_to_third true for every row under that play_id
extra_base['first_to_third'] = extra_base.groupby('play_id')['first_to_third'].transform('max')

#Example play (only case for this particular game)
extra_base.iloc[92:102]

## This is the only particular play where a runner went first to third, you can see in player_position and event_code, it was a base knock to CF
## which makes sense. It was thrown to the shortstop and the play ended, so there was no relay, it was a "cut" play.

,game_str,play_id,timestamp,home_team,away_team,at_bat,play_per_game,top_bottom_inning,pitcher,catcher,...,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,player_position,event_code,first_to_third
92,y1_d070_ACN_QEA,219,7245887,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,10,4,True
93,y1_d070_ACN_QEA,219,7247387,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,255,16,True
94,y1_d070_ACN_QEA,219,7247987,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,255,16,True
95,y1_d070_ACN_QEA,219,7248637,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,255,16,True
96,y1_d070_ACN_QEA,219,7249087,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,255,16,True
97,y1_d070_ACN_QEA,219,7249787,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,8,2,True
98,y1_d070_ACN_QEA,219,7251837,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,8,3,True
99,y1_d070_ACN_QEA,219,7253887,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,6,2,True
100,y1_d070_ACN_QEA,219,7254037,QEA,ACN,52,219,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1416,ACN-1623,NaN,NaN,0,5,True
101,y1_d070_ACN_QEA,227,7477837,QEA,ACN,54,227,top,QEA-0195,QEA-0120,...,QEA-0424,QEA-0365,QEA-0249,ACN-1147,ACN-1180,NaN,ACN-1623,10,4,False


## Second or First to Home Filtering

In [151]:
player_pos = pd.read_csv('player_pos/QEA/ACN/year_1/day_070/player_pos.csv')

#prepping to merge runners_on_bbip for runners on either first or second, with player_pos
first_or_second = runners_on_bbip[(runners_on_bbip['first_baserunner'].notna()) | (runners_on_bbip['second_baserunner'].notna())]

first_or_second_ppos = pd.merge(first_or_second, player_pos, on = ['game_str', 'play_id', 'timestamp'], how = 'inner')

#RENAMING PLAYER_POSITION COLUMNS DUE TO MERGING
#Renamed player_position_x (player_pos from game_info df that specifies each play) into event_player_pos
#Renamed player_position_y (player_pos from player_pos df for each timeframe) into player_position
first_or_second_ppos.rename(columns = {'player_position_x': 'event_player_pos', 'player_position_y': 'player_position'}, inplace=True)
first_or_second_ppos.head(50)

,game_str,play_id,timestamp,home_team,away_team,at_bat,play_per_game,top_bottom_inning,pitcher,catcher,...,batter,first_baserunner,second_baserunner,third_baserunner,event_player_pos,event_code,first_to_third,player_position,field_x,field_y
0,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,1,-0.2790,55.1760
1,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,2,-3.1074,-3.4962
2,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,3,57.7971,64.5051
3,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,4,22.1529,137.5215
4,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,5,-64.2399,97.0239
5,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,6,-31.7448,137.2203
6,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,7,-130.2312,261.8076
7,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,8,-19.1775,299.3532
8,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,9,125.1318,254.2926
9,y1_d070_ACN_QEA,14,282987,QEA,ACN,2,14,top,QEA-0175,QEA-0120,...,ACN-1416,ACN-1623,NaN,NaN,10,4,False,10,-3.3900,0.5163


## Current Status:

During the second webinar, they said that the timestamp data could potentially be messy. It seems that in the above df, the game_info timestamp isn't lining up with the player_pos timestamp such that it includes all the rows from the player_pos df. 

Right now, for the first play in the above query, it looks like player_position #11 (runner on first) goes from (49.72, 79.95) to (32.2, 95.48) in the next frame and it gets more drastic from there. 

If this doesn't change when the new data comes in, we'll know it's on this end. 

### Next Steps:

Goals:
- Query for plays where the runner on second or first goes home
    - Don't need to exclude plays such as: R2 -> Home & R1 -> 3rd
- Merge first_to_third and the above described datasets to use as training data for our continuous extra base safe probability model. 
    - Should train on those scenarios (only scenarios we care about and only scenarios where runners on base and they take an extra base)

Continue to work on this query to find plays where the runner on 1st or 2nd attempts to score. Because there's no indicator that the runner crossed home plate, we'll need to use player_pos data to estimate when the runner crosses home, if they're safe or out, and what the threshold is on the baseline for "attempting to score". 

Ideas: 
- If the ball reaches a certain radius before the runner reaches a certain radius, they're out and vice versa. 
- If the runner crosses a certain (x, y) threshold, they'll be considered to be attempting to score.